In [1]:
from pytrends.request import TrendReq
import datetime

# Login to Google. Only need to run this once, the rest of requests will use the same session.
pytrend = TrendReq()

In [2]:
def getWeeks():

    dStart = input("Enter date of start epidemiological week in DD-MM-YYYY format: ")
    dStart = [int(i) for i in dStart.split("-")]
    start = datetime.datetime(dStart[2],dStart[1],dStart[0])
    
    dEnd = input("Enter date of end epidemiological week in DD-MM-YYYY format: ")
    dEnd = [int(i) for i in dEnd.split("-")]
    end = datetime.datetime(dEnd[2],dEnd[1],dEnd[0])
    
    nextWeek = getNextWeek(start)
    weeks = [start]
    
    while nextWeek <= end:
        if(nextWeek < datetime.datetime.today()):
            weeks.append(nextWeek)
            nextWeek = getNextWeek(nextWeek)
    
    return weeks

In [3]:
def getNextWeek(timeframe):
    return timeframe+datetime.timedelta(days=7)

In [4]:
def getYMD(timeframe):
    return "{}/{}/{}".format(timeframe.day,timeframe.month,timeframe.year)

In [5]:
keyword = input("Enter keyword: ")

Enter keyword: zika sintomas


In [6]:
year = input("Enter year: ")

Enter year: 2017


In [7]:
countryCode = input("Enter country code: ")

Enter country code: MX


In [8]:
weeks = getWeeks()

Enter date of start epidemiological week in DD-MM-YYYY format: 07-01-2017
Enter date of end epidemiological week in DD-MM-YYYY format: 30-12-2017


In [20]:
with open("{}/google/{}-{}.csv".format(year, countryCode, keyword), "w") as file:
    weekEpiArray = []
    cities = {}
    file.write("CITY,")
    file.write(",".join(map(getYMD, weeks))+"\n")
    for i in range(len(weeks)):
        week = weeks[i]
        nextWeek = getNextWeek(week)
        year = week.year
        month = week.month
        day = week.day
        nextWeekC = nextWeek - datetime.timedelta(days=1)
        timeframe = "{}-{}-{} {}-{}-{}".format(year,month,day,nextWeekC.year,nextWeekC.month,nextWeekC.day)
        print("{}-{}-{} -> {}-{}-{}".format(year,month,day,nextWeekC.year,nextWeekC.month,nextWeekC.day))
        pytrend.build_payload(kw_list=[keyword], geo=countryCode, timeframe=timeframe)
        results = pytrend.interest_by_region(resolution="CITY")
        # if a city's search index is 0 it doesn't appear on results, so we have to keep track of it
        tempWeekArray = {}
        for index, row in results.iterrows():
            tempWeekArray[index] = row[keyword]
            if index not in cities:
                cities[index] = index #Store in dictionary to get O(1) access
        weekEpiArray.append(tempWeekArray)
        print("DONE", week)
        
    # Iterate on weekEpiArray and replace non existent cities with zeroes
    searchesByCity = {}
    for city in cities:
        searchesByCity[city] = []
        for week in weekEpiArray:
            if city not in week:
                searchesByCity[city].append(0)
            else:
                searchesByCity[city].append(week[city])
                
    # Finally write to file
    for city in sorted(searchesByCity.keys()):
        file.write("{},".format(city))
        file.write(",".join(map(str,searchesByCity[city])))
        file.write("\n")

2017-1-7 -> 2017-1-13
DONE 2017-01-07 00:00:00
2017-1-14 -> 2017-1-20
DONE 2017-01-14 00:00:00
2017-1-21 -> 2017-1-27
DONE 2017-01-21 00:00:00
2017-1-28 -> 2017-2-3
DONE 2017-01-28 00:00:00
2017-2-4 -> 2017-2-10
DONE 2017-02-04 00:00:00
2017-2-11 -> 2017-2-17
DONE 2017-02-11 00:00:00
2017-2-18 -> 2017-2-24
DONE 2017-02-18 00:00:00
2017-2-25 -> 2017-3-3
DONE 2017-02-25 00:00:00
2017-3-4 -> 2017-3-10
DONE 2017-03-04 00:00:00
2017-3-11 -> 2017-3-17
DONE 2017-03-11 00:00:00
2017-3-18 -> 2017-3-24
DONE 2017-03-18 00:00:00
2017-3-25 -> 2017-3-31
DONE 2017-03-25 00:00:00
2017-4-1 -> 2017-4-7
DONE 2017-04-01 00:00:00
2017-4-8 -> 2017-4-14
DONE 2017-04-08 00:00:00
2017-4-15 -> 2017-4-21
DONE 2017-04-15 00:00:00
2017-4-22 -> 2017-4-28
DONE 2017-04-22 00:00:00
2017-4-29 -> 2017-5-5
DONE 2017-04-29 00:00:00
2017-5-6 -> 2017-5-12
DONE 2017-05-06 00:00:00
2017-5-13 -> 2017-5-19
DONE 2017-05-13 00:00:00
2017-5-20 -> 2017-5-26
DONE 2017-05-20 00:00:00
2017-5-27 -> 2017-6-2
DONE 2017-05-27 00:00:00
201